In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

%%bash
apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
pip install mecab-python3

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("data/QA.csv")

In [4]:
df = df[["title", "question", "answer"]]

In [5]:
df.head()

,title,question,answer
0,Ｑ＆Ａ,本当に国土交通省で行っている調査ですか？,統計法（平成１９年法律第５３号）に基づく「自動車輸送統計調査規則」（昭和３５年運輸省令第１５...
1,Ｑ＆Ａ,どうして私の車が調査に当たったのですか？,日本国内を走る車の中から無作為に調査をお願いしています。\r → 国内の全ての自...
2,Ｑ＆Ａ,どうして私の事業所が調査に当たったのですか？,保有車両数が一定以上である全ての事業所と無作為に抽出した事業所に調査をお願いしています。\r...
3,Ｑ＆Ａ,プライバシーは守られるのですか？,統計法により、必ず守られます。\r → この調査は、統計法（平成１９年法律第５３...
4,Ｑ＆Ａ,この統計はどう使われているのですか？,調査の集計結果は、一例として以下のとおり活用されています。\r → １． 交通政...


In [6]:
import re

In [7]:
import unicodedata

In [8]:
# 全角半角統一
answers = map(lambda text: unicodedata.normalize("NFKC", text), df["answer"])

# 目立つノイズを抑える
answers = map(lambda text: text.replace("\u3000", "").replace("○", " ").replace("→", " ").replace("※", " ※"), answers)

# 連続するスペースを除去
answers = map(lambda text: " ".join(text.split()), answers)

# 文で切る
answers = map(lambda text: "。 ".join(text.split("。")).split(), answers)

def concat_short_sentence(lines):
    """
    短すぎる文は記号と解釈して次の文と繋げる
    """
    
    connected = []
    carry = None
    for line in lines:
        if len(line) > 3:
            if carry is not None:
                connected.append(carry + " " + line)
                carry = None
            else:
                connected.append(line)
        else:
            if carry is not None:
                carry = carry + " " + line
            else:
                carry = line
    else:
        if carry is not None:
            connected.append(carry)
    return connected
    
answers = list(map(lambda lines: concat_short_sentence(lines), answers))

In [9]:
answers[:10]

[['統計法(平成19年法律第53号)に基づく「自動車輸送統計調査規則」(昭和35年運輸省令第15号)により、国土交通省が毎月調査を行っています。',
  '国土交通大臣が選定する自動車の使用者に調査票を配布し、指定された期間の輸送状況等をご記入いただくという方法で調査を行っています。',
  'なお、調査方法の変更により、調査票の配布・回収は郵送で行っています。'],
 ['日本国内を走る車の中から無作為に調査をお願いしています。',
  '国内の全ての自動車を毎月調査することは難しいため、地域別、車種別の層ごとに無作為に抽出した自動車の使用者の方に調査へのご協力をお願いしています。'],
 ['保有車両数が一定以上である全ての事業所と無作為に抽出した事業所に調査をお願いしています。',
  '貨物事業所への調査については、保有車両数が一定以上である全ての事業所の皆様と、地域別、保有車両数別の層ごとに無作為に抽出した事業所の皆様に調査へのご協力をお願いしています。'],
 ['統計法により、必ず守られます。',
  'この調査は、統計法(平成19年法律第53号)に基づき政府が実施する基幹統計調査です。',
  '本調査において知り得た事項については、統計法第41条により守秘義務が課せられており、秘密の保護には万全を期しています。'],
 ['調査の集計結果は、一例として以下のとおり活用されています。', '1. 交通政策等策定の基礎資料', '2. モーダルシフト推進のための基礎資料'],
 ['調査対象は車両1台又は事業所ごととなっています。',
  '【調査対象が自動車の場合】',
  '車両1台ごとを調査対象にしているため、複数の車両を所有されている場合は短期間に再度調査へのご協力をお願いすることがあります。',
  'また、一度調査対象になった車両は、営業用自動車の場合は1年以内、自家用自動車の場合は3年以内に再度調査をお願いすることは原則としてありません。',
  'しかし、地域別・車種別の層によっては、車両数が少なく短期間に再度調査へのご協力をお願いすることがあります。',
  '【調査対象が事業所の場合】',
  '保有車両数が一定以上である事業所につきましては、毎月、調査にご協力をお願いしています。',
  '無作為に抽出した事業所につきましては、

In [10]:
answers[-10:]

[['就労準備支援モデル事業の利用に当たっては、自立相談支援機関による一連の支援プロセス(プラン作成、支援調整会議等)、行政による支援決定を経ることが必要である。',
  'これらの取扱いは生活保護受給者についても基本的には同様であるが、生活保護受給者に対する支援は福祉事務所のケースワーカーが責任を負うものであることから、一義的には事業利用の必要性をケースワーカーが判断した上で、自立相談支援機関と必要な調整を行い、本モデル事業における一連のプロセスを行う必要がある。',
  'なお、この場合、福祉事務所が把握している当該受給者に関する情報等については、可能な限り自立相談支援機関と共有できるようにするなど、円滑な支援開始のため適切な連携を図っていただくようご留意願いたい。'],
 ['可能である。'],
 ['平成25年度におけるモデル事業の補助対象経費については、給料、職員手当等、報酬、共済費、報償費、旅費、賃金、需用費(消耗品費、燃料費、印刷製本費、修繕費、食糧費)、使用料、賃貸料、役務費(通信運搬費、保険料、手数料)、委託費、備品購入費(価格30万円以上の備品を除く)、負担金としている。',
  '就労訓練事業の推進モデル事業においては、・担い手となる法人等への啓発・研修・地域における就労訓練事業に関する調査研究や協議会の開催・就労訓練事業者に対する立ち上げ支援等の事業に取り組むことができることとしており、これらの対象経費のうち必要な支出を検討されたい。'],
 ['「生活困窮者自立促進支援モデル事業実施要領」第4の4の事業として、補助対象となり得るものと考えられるが、教育担当部局とも十分連携の上、事業内容を検討されたい。'],
 ['「生活困窮者自立支援制度施行円滑化特別対策事業」については、平成26年度にモデル事業を実施しない自治体についても補助対象とすることとしている。'],
 ['お見込みのとおり。'],
 ['モデル事業の補助協議にあたって、個別具体的な減額基準は定めていないが、補助基準額は12ヶ月の実施期間を想定して設定しているものであるため、実際の事業実施期間などを勘案して協議を行うこととしている。'],
 ['予算が単年度主義であることから、単年度契約が基本になると考えている。'],
 ['今後、モデル事業実施自治体を対象にアンケート調査等を実施

In [28]:
import MeCab

In [29]:
tagger = MeCab.Tagger("-Owakati")

In [36]:
tokenized = []
for lines in answers:
    t = []
    for line in lines:
        t.append(tagger.parse(line).strip())
    tokenized.append(" ".join(t))

In [37]:
tokenized[:10]

['統計 法 ( 平成 19 年 法律 第 53 号 ) に 基づく 「 自動車 輸送 統計 調査 規則 」 ( 昭和 35 年 運輸省 令 第 15 号 ) により 、 国土 交通省 が 毎月 調査 を 行っ て い ます 。 国土 交通 大臣 が 選定 する 自動車 の 使用 者 に 調査 票 を 配布 し 、 指定 さ れ た 期間 の 輸送 状況 等 を ご 記入 いただく という 方法 で 調査 を 行っ て い ます 。 なお 、 調査 方法 の 変更 により 、 調査 票 の 配布 ・ 回収 は 郵送 で 行っ て い ます 。',
 '日本 国内 を 走る 車 の 中 から 無作為 に 調査 を お願い し て い ます 。 国内 の 全て の 自動車 を 毎月 調査 する こと は 難しい ため 、 地域 別 、 車種 別 の 層 ごと に 無 作為 に 抽出 し た 自動車 の 使用 者 の 方 に 調査 へ の ご 協力 を お願い し て い ます 。',
 '保有 車両 数 が 一定 以上 で ある 全て の 事業 所 と 無作為 に 抽出 し た 事業 所 に 調査 を お願い し て い ます 。 貨物 事業 所 へ の 調査 について は 、 保有 車両 数 が 一定 以上 で ある 全て の 事業 所 の 皆様 と 、 地域 別 、 保有 車両 数 別 の 層 ごと に 無 作為 に 抽出 し た 事業 所 の 皆様 に 調査 へ の ご 協力 を お願い し て い ます 。',
 '統計 法 により 、 必ず 守ら れ ます 。 この 調査 は 、 統計 法 ( 平成 19 年 法律 第 53 号 ) に 基づき 政府 が 実施 する 基幹 統計 調査 です 。 本 調査 において 知り 得 た 事項 について は 、 統計 法 第 41 条 により 守秘 義務 が 課せ られ て おり 、 秘密 の 保護 に は 万全 を 期し て い ます 。',
 '調査 の 集計 結果 は 、 一 例 として 以下 の とおり 活用 さ れ て い ます 。 1 . 交通 政策 等 策定 の 基礎 資料 2 . モーダルシフト 推進 の ため の 基礎 資料',
 '調査 対象 は 車両 1 台 又は 事業 所 ごと と なっ て い 

In [38]:
df["answer"] = tokenized

In [40]:
df.head()

,title,question,answer
0,Ｑ＆Ａ,本当に国土交通省で行っている調査ですか？,統計 法 ( 平成 19 年 法律 第 53 号 ) に 基づく 「 自動車 輸送 統計 調...
1,Ｑ＆Ａ,どうして私の車が調査に当たったのですか？,日本 国内 を 走る 車 の 中 から 無作為 に 調査 を お願い し て い ます 。 ...
2,Ｑ＆Ａ,どうして私の事業所が調査に当たったのですか？,保有 車両 数 が 一定 以上 で ある 全て の 事業 所 と 無作為 に 抽出 し た ...
3,Ｑ＆Ａ,プライバシーは守られるのですか？,統計 法 により 、 必ず 守ら れ ます 。 この 調査 は 、 統計 法 ( 平成 19...
4,Ｑ＆Ａ,この統計はどう使われているのですか？,調査 の 集計 結果 は 、 一 例 として 以下 の とおり 活用 さ れ て い ます ...


In [52]:
df_dropped = df.dropna()

In [53]:
titles = []
questions = []
for i, (title, question) in enumerate(zip(df_dropped["title"], df_dropped["question"])):
    title = unicodedata.normalize("NFKC", title)
    question = unicodedata.normalize("NFKC", question)
    titles.append(tagger.parse(title).strip())
    questions.append(tagger.parse(question).strip())

In [54]:
df_dropped["title"] = titles
df_dropped["question"] = questions

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [55]:
df_dropped.head()

,title,question,answer
0,Q & A,本当に 国土 交通省 で 行っ て いる 調査 です か ?,統計 法 ( 平成 19 年 法律 第 53 号 ) に 基づく 「 自動車 輸送 統計 調...
1,Q & A,どうして 私 の 車 が 調査 に 当たっ た の です か ?,日本 国内 を 走る 車 の 中 から 無作為 に 調査 を お願い し て い ます 。 ...
2,Q & A,どうして 私 の 事業 所 が 調査 に 当たっ た の です か ?,保有 車両 数 が 一定 以上 で ある 全て の 事業 所 と 無作為 に 抽出 し た ...
3,Q & A,プライバシー は 守ら れる の です か ?,統計 法 により 、 必ず 守ら れ ます 。 この 調査 は 、 統計 法 ( 平成 19...
4,Q & A,この 統計 は どう 使わ れ て いる の です か ?,調査 の 集計 結果 は 、 一 例 として 以下 の とおり 活用 さ れ て い ます ...


In [59]:
df_dropped.to_csv("data/QA.normalized.csv", encoding="utf-8")